# Explorando las formaciones frente a errores en la percepción
## A partir de "Maneuvering and robustness issues in undirecter displacemente-consensus-based formation control" de Héctor García de Marina
En el artículo se modelan los errores en la percepción como un error de escalado y una matriz de rotación que representan un alineado incorrecto en la medida de las posiciones.
$$ (p_i-p_j)_{measurement}=aR(p_i-p_j)$$
$$ a \in \mathcal{R}, R \in \mathcal{SO}(m)$$

### Carga de módulos necesarios

In [1]:
import RobotModels as rob
import numpy as np
import matplotlib.pyplot as plt
import graph_utils as graf
import time
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
import robot_plot_utils as rpu

## Formación inicial con 2 agentes
Generación de las posiciones iniciales y el grafo.

Estamos trabajando con distance-based-formation control, es decir la información que tienen los agentes es la posición relativa de sus vecinos respecto de ellos mismos.

Pruebo primer el caso sin meter errores para ver que todo funciona con dos agentes. El objetivo es que el agente 1 se mantenga a 5m (en eje x del agente 2)

In [2]:
#Posiciones iniciales
pi=np.array([[-15,0],[15,0]])
p=np.zeros([2,2])
numnodos,m=np.shape(pi)
#Formacion
#zstar=np.array([-5,-5])
#Grafo
Z=np.array([[1,2]])
numarcos,m=np.shape(Z)
n,m= np.shape(pi)
for i in range(n):
    plt.plot(pi[i,0],pi[i,1],'o')

rpu.dibuja_grafo_2D(pi,Z)

Im=np.eye(2)
pesos=0.5*np.ones([numnodos,numnodos])
L=graf.matriz_laplaciana(Z,pesos,numnodos,numarcos)

Lbar=np.kron(L,Im)
print("Lbar: ")
print(Lbar)

pstar_s=np.array([-5 ,0 ,5 ,0]);
pstar=np.array([[-5 ,0] ,[5 ,0]]);
t=0
tf=100
dt=0.5
nptos=(int)((tf-t)/dt)
#print(nptos)
fig2 =plt.figure()
p=np.zeros([2,2])
p=pi
posiciones=np.zeros([2*n , nptos])
posiciones[:,0]=graf.stack(pi)
k=0
print( p)
pmeasured=np.zeros([2,2])




for k in range(nptos-1):
    pmeasured[0,:]=p[0,:]-p[1,:]
    #pmeasured[1,:]=p[0,:]-p[1,:]+b[1,0]*np.ones([1,2])
    pmeasured[1,:]=p[1,:]-p[0,:]
    pmeasured_s=graf.stack(pmeasured)
    #print("Pmeasured:")
    #print(pmeasured_s)   
    pdot=-np.dot(Lbar,pmeasured_s-pstar_s)
    #print("Pdot: ")
    #print(pdot)
    l=0
    for i in range(0,numnodos,1):
        #print(pdot[l])
        #print(pdot[l+1])
        xdot,ydot,thetadot=rob.integrator_point_model(pdot[l],pdot[l+1])
        p[i,0]+=dt*xdot
        p[i,1]+=dt*ydot
        l+=2
        #print(p[i,:])
    
    p_s=graf.stack(p)
    posiciones[:,k+1]=p_s[:]
    t+=dt
print(posiciones[:,k])
colors_f=['bo','go','ro','co','yo','mo','ko','bo','go']
colors_i=['bx','gx','rx','cx','yx','mx','kx','bx','gx']
colors_l=['b','g','r','c','y','m','k','b:','g:']

j=0
for i in range(numnodos):
    plt.plot(posiciones[j,0],posiciones[j+1,0],colors_i[i])
    plt.plot(posiciones[j,nptos-1],posiciones[j+1,nptos-1],colors_f[i])
    plt.plot(posiciones[j,:],posiciones[j+1,:],colors_l[i])
    j=j+2
plt.grid()

Lbar: 
[[ 0.5  0.  -0.5 -0. ]
 [ 0.   0.5 -0.  -0.5]
 [-0.5 -0.   0.5  0. ]
 [-0.  -0.5  0.   0.5]]


ValueError: could not broadcast input array from shape (4,1) into shape (4,)

Repetimos con más agentes (4) pero sin introducir aún ningún error

In [65]:
#Posiciones iniciales
pi=np.array([[-15,0],[15,0], [-15, 10],[15,10]])
numnodos,m=np.shape(pi)
p=np.zeros([numnodos,2])
#Formacion
#zstar=np.array([-5,-5])
#Grafo
Z=np.array([[1,2],[2,3],[3,4]])
numarcos,m=np.shape(Z)

for i in range(numnodos):
    plt.plot(pi[i,0],pi[i,1],'o')

rpu.dibuja_grafo_2D(pi,Z)


Im=np.eye(2)
pesos=0.5*np.ones([numnodos,numnodos])
L=graf.matriz_laplaciana(Z,pesos,numnodos,numarcos)

Lbar=np.kron(L,Im)
print("Lbar: ")
print(Lbar)


pstar=np.array([[1,0],[1,1],[1,0]]);
pstar_s=graf.stack(pstar)

t=0
tf=100
dt=0.5
nptos=(int)((tf-t)/dt)
#print(nptos)
fig2 =plt.figure()
p=np.zeros([numnodos,m])
p=pi
posiciones=np.zeros([2*numnodos , nptos])
posiciones[:,0]=graf.stack(pi)
k=0
print( p)
pmeasured=np.zeros([numarcos,m])
nptos=3
# TODO: Revisar porque las posiciones que tenemos son las que cada agente mide con sus vecinos. 
# Se puede encapsular??
for k in range(nptos-1):
    for j in range(0,numarcos):
        head=Z[j,0]
        tail=Z[j,1]
        pmeasured[head-1,:]=p[head-1,:]-p[tail-1,:]
        #pmeasured[tail-1,:]=-pmeasured[head-1,:]
        print(head)
        print(tail)
        print(pmeasured)
    pmeasured_s=graf.stack(pmeasured)
    print("Pmeasured:")
    print(pmeasured_s)   
    pdot=-np.dot(Lbar,pmeasured_s-pstar_s)
    print("Pdot: ")
    print(pdot)
    l=0
    for i in range(0,numnodos,1):
        #print(pdot[l])
        #print(pdot[l+1])
        xdot,ydot,thetadot=rob.integrator_point_model(pdot[l],pdot[l+1])
        p[i,0]+=dt*xdot
        p[i,1]+=dt*ydot
        l+=2
        #print(p[i,:])
    
    p_s=graf.stack(p)
    posiciones[:,k+1]=p_s[:]
    t+=dt
print(pmeasured_s-pstar_s)
colors_f=['bo','go','ro','co','yo','mo','ko','bo','go']
colors_i=['bx','gx','rx','cx','yx','mx','kx','bx','gx']
colors_l=['b','g','r','c','y','m','k','b:','g:']

j=0
for i in range(numnodos):
    plt.plot(posiciones[j,0],posiciones[j+1,0],colors_i[i])
    plt.plot(posiciones[j,nptos-1],posiciones[j+1,nptos-1],colors_f[i])
    plt.plot(posiciones[j,:],posiciones[j+1,:],colors_l[i])
    j=j+2
plt.grid()

Lbar: 
[[ 0.5  0.  -0.5 -0.   0.   0.   0.   0. ]
 [ 0.   0.5 -0.  -0.5  0.   0.   0.   0. ]
 [-0.5 -0.   1.   0.  -0.5 -0.   0.   0. ]
 [-0.  -0.5  0.   1.  -0.  -0.5  0.   0. ]
 [ 0.   0.  -0.5 -0.   1.   0.  -0.5 -0. ]
 [ 0.   0.  -0.  -0.5  0.   1.  -0.  -0.5]
 [ 0.   0.   0.   0.  -0.5 -0.   0.5  0. ]
 [ 0.   0.   0.   0.  -0.  -0.5  0.   0.5]]
[[-15   0]
 [ 15   0]
 [-15  10]
 [ 15  10]]
1
2
[[-30.   0.]
 [  0.   0.]
 [  0.   0.]]
2
3
[[-30.   0.]
 [ 30. -10.]
 [  0.   0.]]
3
4
[[-30.   0.]
 [ 30. -10.]
 [-30.   0.]]
Pmeasured:
[-30.   0.  30. -10. -30.   0.]


ValueError: shapes (8,8) and (6,) not aligned: 8 (dim 1) != 6 (dim 0)